In [ ]:
import numpy as np
import numpy.linalg as npl
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.cm as cmp
import scipy.integrate as sc
#defining constants

c=3*10**8
n=1000         
p=2          
snb=31    

#Gaussian distribution and same standard deviations for both omega matter and h
sigo=0.01          #standard deviation for omega matter
sigh=0.01           #standard deviation of h

ar=np.empty([n,p+1])     
z=[]                        
M=[]                        

Cov=open('jla_mub_covmatrix.txt','r')
data=open('jla_mub_0.txt','r')


l1=1
C=np.loadtxt(Cov,skiprows=1)
C=np.reshape(C,(s,s))
cinv=npl.inv(C)          


i=1
for line in data:
    a=[0,0]
    a=line.split()
    if i==1:
        i=i+1
        continue
    else:
        z.append(float(a[0]))
        M.append(float(a[1]))

#defining functions
def eta(a,om):                      #eta
    if om>=0.9999:
        om=0.9999
    elif om<=0.0:
        om=0.000001
    else:
        pass
    s=((1.0-om)/om)**(1.0/3.0)
    n= 2.0*(np.sqrt((s*s*s)+1))*(((a**(-4.0))-(0.1540*s*(a**(-3.0)))+(0.4304*s*s*(a**(-2.0)))+(0.19097*s*s*s*(a**(-1.0)))+(0.066941*s*s*s*s)))**(-1.0/8.0)
    return n

def Dl(z,om):                         
    eta1=eta(1,om)
    eta2=eta(1/(1+z),om)
    d=(3000.0*(1+z))*(eta1-eta2)
    return d

def mu(z,om,h):                     
    d=Dl(z,om)
    m=25-(5*np.log10(h))+(5*np.log10(d))
    return m

diff=np.empty(s)  
def likelihood(om,h,z,M):                   
    if om<=0.0 or h<=0.0:               
        l=-1.e100
    else:
        for i in range(s):
            diff[i]=M[i]-mu(z[i],om,h)
        dt=np.dot(cinv,diff)
        l=-0.5*np.dot(np.transpose(diff),dt)      #dot products
    return l    #l is ln(L)


ar[0,0]=np.random.uniform() 
ar[0,1]=np.random.uniform()
ar[0,2]=likelihood(ar[0,0],ar[0,1],z,M)
accn=0

#Sampling:
for i in range(1,n):
    lpre=ar[i-1,2]
    omnext=np.random.normal(ar[i-1,0],sigo)    
    hnext=np.random.normal(ar[i-1,1],sigh)
    lnext=likelihood(omnext,hnext,z,M)


    if lnext>=lpre:
        ar[i,0]=omnext
        ar[i,1]=hnext
        ar[i,2]=lnext
        accn=accn+1
        print("Accepted with higher likelihood.")
    else:
        x=np.random.uniform()
        if (lnext-lpre)>np.log(x):
            ar[i,0]=omnext
            ar[i,1]=hnext
            ar[i,2]=lnext
            accn=accn+1
            print("Accepted with lesser likelihood.")
        else:
            ar[i,0]=ar[i-1,0]
            ar[i,1]=ar[i-1,1]
            ar[i,2]=lpre
            print("rejected")

#burnin
r=n//25

#calculating statistical values:
omm=np.mean(ar[r:,0])         #mean or estimated value of omega matter
hm=np.mean(ar[r:,1])          #mean or estimated value of h
omsd=np.std(ar[r:,0])         #standard deviation in omega matter
hsd=np.std(ar[r:,1])           #standard deviation in h
accr=(accn*100)/n                    #acceptance ratio


print('Estimated value of omega matter is:'+str(omm))
print('Estimated standard deviation in omega matter is:'+str(omsd))
print('estimated value of scaling factor h is:'+str(hm))
print('Estimated standard deviation in scaling factor is:'+str(hsd))
print('Percentage of samples accepted (acceptance ratio) is:'+str(accr))

#Calculating theoretical values of distance modulus from our estimates
yth=np.empty(s)
for i in range(s):
    yth[i]=mu(z[i],omm,hm)

#Plotting:

plt.figure(1)
plt.xlabel('Redshift z')
plt.ylabel('Distance modulus mu')
plt.title('Distance Modulus vs redshift data comparison.')
plt.plot(z,M,c='red',label="Given Data")
plt.plot(z,yth,c='blue',label="Theoretical Data from estimates obtained")
plt.legend()
plt.show()
#2. Scatter plot:
plt.figure(2)
plt.xlabel('Redshift z')
plt.ylabel('Given distance modulus mu')
plt.title('Scatter Plot of Sampled data post burn in')
plt.scatter(ar[r:,0],ar[r:,1],c=-ar[r:,2],marker='x')
plt.xlim(0.1,0.5)
plt.ylim(0.6,0.8)
plt.xlabel('Omega Matter')
plt.ylabel('Scaling Factor h')
#3: histogram
plt.figure(3)
plt.xlabel('omega matter')
plt.ylabel('counts')
plt.hist(ar[r:,0],bins=10)
plt.figure(4)
plt.xlabel("h")
plt.ylabel("Counts")
plt.hist(ar[r:,1],bins=10)
plt.show()
